This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Implement a trie with find, insert, remove, and list_words methods.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Can we assume we are working with strings?
    * Yes
* Are the strings in ASCII?
    * Yes
* Should `find` only match exact words with a terminating character?
    * Yes
* Should `list_words` only return words with a terminating character?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

<pre>

         root
       /  |  \
      h   a*  m
     / \   \   \
    a   e*  t*  e*
   / \         / \
  s*  t*      n*  t*
             /
            s*

find

* Find on an empty trie
* Find non-matching
* Find matching

insert

* Insert on empty trie
* Insert to make a leaf terminator char
* Insert to extend an existing terminator char

remove

* Remove me
* Remove mens
* Remove a
* Remove has

list_words

* List empty
* List general case
</pre>



## Algorithm

Refer to the [Solution Notebook](http://nbviewer.ipython.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/trie/trie_solution.ipynb).  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

## Code

In [1]:
from collections import OrderedDict


class Node(object):

    def __init__(self, data, parent=None, terminates=False):
        self.data = data
        self.parent = parent
        self.terminates = terminates
        self.children = []
        
    def __repr__(self):
        return self.data

class Trie(object):

    def __init__(self):
        self.root = Node('')
        
    def find(self, word, return_node=False):
        if len(word) == 0:
            return None
        return self._find(word, self.root, return_node=return_node)
        
    def _find(self, word, node, return_node=False):
        for c in node.children:
            if word[0] == c.data:
                if len(word) == 1:
                    if not return_node:
                        return c.terminates if c.terminates else None
                    else:
                        return c if c.terminates else None
                return self._find(word[1:], c, return_node=return_node)
#         return False
        return None

    def insert(self, word):
        assert self.root.data == ''
        current = self.root
        word_len = len(word)
        for idx, char in enumerate(word):
            added = False
            for child in current.children:
                if child.data == char:
                    added = True
                    current = child
                    break
            if not added:  # not in the tree... add manually
                char_node = Node(char, parent=current)
                current.children.append(char_node)
                current = char_node
            if idx == word_len - 1:
                current.terminates = True

    def remove(self, word):
        assert len(word) > 0
        if not self.find(word):
            return None
        word_node = self.find(word, return_node=True)

        # trace up, deleting nodes, until either
        # 1. a terminating node is found
        # 2. a node has children)
        # Then:    break
        
        current = word_node
        current.terminates = False
        at_tip_of_word = True
        while current is not None and current.data != '':  # while current exists and its not the root node
            if len(current.children) >= 1 or current.terminates:  # at_tip_of_word
                break
            # delete node
            # find current in current.parent
            assert current.parent is not None  # current is not the root node
            for idx, c in enumerate(current.parent.children):
                if current == c:
                    current.parent.children.pop(idx)
                    break

            current = current.parent
            at_tip_of_word = False

    def list_words(self):
        # This is a BFS with path backtracking via the .parent attribute
        results = []
        q = [self.root]
        while len(q):
            current = q.pop()
            if current.terminates:
                results.append(self.backtrack(current))
            
            for child in current.children:
                q.append(child)
        return results

    def backtrack(self, node):
        result = ''
        while node is not None:
            result += node.data
            node = node.parent
        return result[::-1]

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [2]:
# %load test_trie.py
from nose.tools import assert_true


class TestTrie(object):

    def test_trie(self):
        print('Test: Remove from empty trie')
        trie = Trie()
        assert_true(trie.remove('foo') is None)

        print('Test: Insert')
        words = ['a', 'at', 'has', 'hat', 'he',
                 'me', 'men', 'mens', 'met']
        for word in words:
            trie.insert(word)
        for word in trie.list_words():
            assert_true(trie.find(word) is not None)

        # Remove me
        # Remove mens
        # Remove a
            
        print('Test: Remove me')
        trie.remove('me')
        words_removed = ['me']
        words = ['a', 'at', 'has', 'hat', 'he',
                 'men', 'mens', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Test: Remove mens')
        trie.remove('mens')
        words_removed = ['me', 'mens']
        words = ['a', 'at', 'has', 'hat', 'he',
                 'men', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Test: Remove a')
        trie.remove('a')
        words_removed = ['a', 'me', 'mens']
        words = ['at', 'has', 'hat', 'he',
                 'men', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Test: Remove has')
        trie.remove('has')
        words_removed = ['a', 'has', 'me', 'mens']
        words = ['at', 'hat', 'he',
                 'men', 'met']
        for word in words:
            assert_true(trie.find(word) is not None)
        for word in words_removed:
            assert_true(trie.find(word) is None)

        print('Success: test_trie')


def main():
    test = TestTrie()
    test.test_trie()


if __name__ == '__main__':
    main()

Test: Remove from empty trie
Test: Insert
Test: Remove me
Test: Remove mens
Test: Remove a
Test: Remove has
Success: test_trie


## Solution Notebook

Review the [Solution Notebook](http://nbviewer.ipython.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/trie/trie_solution.ipynb) for a discussion on algorithms and code solutions.